In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from numpy import asarray
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Input,LeakyReLU,Activation,Concatenate,BatchNormalization,Conv2DTranspose
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

C:\Users\Aryan\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Aryan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Aryan\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [7]:
class CGAN:
    @staticmethod
    def discriminator(image_shape):
        init = RandomNormal(stddev = 0.02)  #initializer
        in_image = Input(shape=(image_shape,image_shape,3))
        
        d = Conv2D(64,(4,4),strides=(2,2),padding='same', kernel_initializer=init)(in_image)
        d = LeakyReLU(alpha=0.2)(d)
        
        d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = InstanceNormalization(axis=-1)(d)
        d = LeakyReLU(alpha=0.2)(d)
        
        d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = InstanceNormalization(axis=-1)(d)
        d = LeakyReLU(alpha=0.2)(d)
        
        d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
        d = InstanceNormalization(axis=-1)(d)
        d = LeakyReLU(alpha=0.2)(d)
        
        d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
        d = InstanceNormalization(axis=-1)(d)
        d = LeakyReLU(alpha=0.2)(d)
        
        patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
        
        model = Model(in_image, patch_out)
        
        model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
        return model
    
    def resnet_block(n_filters, input_layer):
        init = RandomNormal(stddev=0.02)

        g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
        g = InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
        g = InstanceNormalization(axis=-1)(g)

        g = Concatenate()([g, input_layer])
        return g

    @staticmethod
    def generator(image_shape, n_resnet=9):
        init = RandomNormal(stddev=0.02)
    
        in_image = Input(shape=(image_shape,image_shape,3))
    
        g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
        g = InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
        g = InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
        g = InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        for _ in range(n_resnet):
            g = CGAN.resnet_block(256, g)

        g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
        g = InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
        g = InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
        g = InstanceNormalization(axis=-1)(g)
        out_image = Activation('tanh')(g)

        model = Model(in_image, out_image)
        return model
    
    @override
    def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
        g_model_1.trainable = True

        d_model.trainable = False

        g_model_2.trainable = False

        input_gen = Input(shape=(image_shape,image_shape,3))
        gen1_out = g_model_1(input_gen)
        output_d = d_model(gen1_out)

        input_id = Input(shape=(image_shape,image_shape,3))
        output_id = g_model_1(input_id)

        output_f = g_model_2(gen1_out)

        gen2_out = g_model_2(input_id)
        output_b = g_model_1(gen2_out)
       
        model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])

        opt = Adam(lr=0.0002, beta_1=0.5)

        model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
        return model